# Behavior cloning for shiftable loads

In [ ]:
using DataFrames, CSV
using BSON: @save, @load
using BSON, NPZ
using TOML
using Flux, MLJ
using StatsBase
using Random
using Plots, Plots.Measures
using Revise, HEMS

ENV["COLUMNS"] = 2000
Random.seed!(1)

In [ ]:
## Plots default
default(; labelfontsize = 13, tickfontsize = 11,
    framestyle = :box,
    margin = 0.5mm,
    legendfontsize=11,
    xticks=:auto
)

## Read expert dataset

In [ ]:
root_dir = "../../"

home_dir = joinpath(root_dir, "data/home/")
IL_data_dir = joinpath(root_dir, "data/learning/IL_homeB")
IL_model_dir = joinpath(root_dir, "model/IL_homeB")
img_dir = joinpath(root_dir, "img")
;

In [ ]:
h = read_home_config(joinpath(home_dir, "homeB.toml"));

In [ ]:
# `d` and `dt` are training set and test set respectively
@load joinpath(IL_data_dir, "train_xy.bson") d

dt = BSON.load(joinpath(IL_data_dir, "test_xy.bson"))[:d];
keys(dt)

## Train agents for shiftable loads
- Train a DNN-based agent 
- Report performance 
- Store the obtained agent

We empirically found that the same set of hyperparameters worked well for all shiftable loads in this study.

In [ ]:
function report_classification_perf(agent, X, y)

    ŷ = vec(agent(X) .>= 0.5) 
    acc = MLJ.accuracy(ŷ, y)
    # cm = MLJ.confusion_matrix(coerce(ŷ, OrderedFactor), coerce(y, OrderedFactor))
    cm = MLJ.confusion_matrix(categorical(ŷ; ordered = true), categorical(y; ordered = true))
    f1 = MLJ.f1score(ŷ, y)
    @show cm acc f1
    nothing
end

In [ ]:
agents = Agent[]
shiftable_loads = [l["id"] for l in Iterators.Flatten((h["SU"], h["SI"]))]
loss_histories = []
shiftable_loads

In [ ]:
@time begin
for id in shiftable_loads
    X, y = d[id]
    Xt, yt = dt[id]
    
    agent, loss_hist = train_SU_agent(X, y; device = Flux.cpu, epochs = 500, η = 0.2e-2,
        report_freq = 0, w1 = 2.5)
    push!(agents, agent)
    push!(loss_histories, loss_hist)
end
end

In [ ]:
wi = 8  # which shiftable load 
loss_hist = loss_histories[wi]
id = shiftable_loads[wi]
ne = size(loss_hist)[1]
@show id ne
plot(; size = (610, 220), xlabel = "Epoch", ylabel = "Loss")
plot!(0:ne-1, loss_hist; label=["train" "validation"], bottom_margin=3.5mm, top_margin=-1mm, left_margin=1.5mm, right_margin=1.5mm)
savefig(joinpath(img_dir, "train_$(id).pdf"))
current()

In [ ]:
# check performance statistics 
for (id, agent) in zip(shiftable_loads, agents)
    @show id
    X, y = d[id]
    Xt, yt = dt[id]
    println("- Training performance")
    report_classification_perf(agent, X, y)
    println("- Test performance")
    report_classification_perf(agent, Xt, yt)
end

In [ ]:
@save joinpath(IL_model_dir, "agents.bson") agents=Dict(id => agent for (id, agent) in zip(shiftable_loads, agents))